<a href="https://colab.research.google.com/github/jjandnn/ReenactGAN/blob/master/ReenactGAN_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 索引

inproceedings{wayne2018reenactgan,

 author = {Wu, Wayne and Zhang, Yunxuan and Li, Cheng and Qian, Chen and Loy, Chen Change},

 title = {ReenactGAN: Learning to Reenact Faces via Boundary Transfer},
 booktitle = {ECCV},

 month = September,

 year = {2018}
}

In [0]:
# 查看设备，确认GPU情况
! /opt/bin/nvidia-smi

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# 将自己之前保存在云盘中的训练模型复制到colab的用户目录里来，如果是初次运行，跳过这一步，云盘的目录视自己的目录结构而定
!cp -r '/content/drive/My Drive/Colab Notebooks/github_projects/ReenactGAN-checkpoints/checkpoints' '/content/checkpoints'

In [0]:
# 下载软件和数据集
!rm -rf sample_data
!git clone https://github.com/jjandnn/ReenactGAN.git
!mv ReenactGAN/* .
!rm -rf ReenactGAN
!pip install dominate
!gdown https://drive.google.com/uc?id=1jQ6d76T5GQuvQH4dq8_Wq1T0cxvN0_xp

# 解压数据集这步在云端挺耗时，要有耐心
!tar xvf CelebV.tar.gz
!rm -rf CelebV.tar.gz

## 训练模型

In [0]:
# 训练边界转换器(指的是脸部的landmark图转换)
# 数据集：CelebV
# --root_dir表示数据集的目录，--which_target的target表示要作为目标的人
# --which_target可选参数范本： 0：伊曼纽尔·马克龙 1：凯瑟琳 2：马云 3：特蕾莎·梅 4：唐纳德·特朗普，sh中默认选的是4，即川普
# --gpu_ids 0 由于colab是k80或p100就一个GPU，所以就默认设备 0 。自己跑本地，有几个排序写几个，如：--gpu_ids 0,1,2,3 代表4个GPU
# 其他参数打开train_Transformer.sh，自行查看

!bash scripts/train_Transformer_colab.sh

2019-11-10 14:53:55] (epoch: 500002, iters: 4700, time: 0.229) D_real_A: 0.065561 D_fake_A: 0.170544 D_real_B: 1.094720 D_fake_B: 0.642012 Cycle_AB: 0.019447 Cycle_BA: 0.025053 GAN_AB: 2.040681 GAN_BA: 3.760077 PCA_AB: 0.094486 PCA_BA: 0.070860 
2019-11-10 14:55:27] (epoch: 500002, iters: 4800, time: 0.228) D_real_A: 0.439216 D_fake_A: 0.262510 D_real_B: 0.580460 D_fake_B: 0.604987 Cycle_AB: 0.020865 Cycle_BA: 0.024671 GAN_AB: 1.973024 GAN_BA: 2.745995 PCA_AB: 0.104680 PCA_BA: 0.098583 
2019-11-10 14:56:59] (epoch: 500002, iters: 4900, time: 0.229) D_real_A: 0.634898 D_fake_A: 0.254840 D_real_B: 1.171419 D_fake_B: 0.519977 Cycle_AB: 0.021298 Cycle_BA: 0.023376 GAN_AB: 1.843452 GAN_BA: 2.769288 PCA_AB: 0.087257 PCA_BA: 0.088804 


In [0]:
# 训练解码器
# 数据集：CelebV
# --root_dir在这里代表目标人物的目录
# 其他参数自行查看train_Decoder.sh

!bash scripts/train_Decoder.sh

In [0]:
# 保存训练模型的进度到谷歌云盘，请根据自己的目录保存
!cp -r '/content/checkpoints' '/content/drive/My Drive/Colab Notebooks/github_projects/ReenactGAN-checkpoints/'

In [0]:
# 复制自己训练的模型到预训练文件夹，日期根据具体模型的时间节点来填写，人物也是
!bash /content/scripts/move_models.sh ./checkpoints/Transformer_2019-xx-xx_xx-xx-xx/G_BA_xx.pth ./checkpoints/Decoder_2019-xx-xx_xx-xx-xx/xx_net_G.pth trump
# 运行测试，生成图片
!bash /content/scripts/test.sh

## 不训练，直接测试软件自带的预训练模型生成图片，查看效果

In [0]:
# test
# 下载单人的边界转换器和编码器预训练模型：川普
!gdown https://drive.google.com/uc?id=1MBWABJK9webZxAMvN9Cl5FBhXateppzu
!gdown https://drive.google.com/uc?id=1v-8kh0N56alKiSoBAENXp9KNJ0lg_Qtq
!mv trump_net_decoder.pth pretrained_models/trumpcelebv_net_decoder.pth
!mv trump_net_transformer.pth pretrained_models/trumptmp2_net_transformer.pth

In [0]:
# 运行测试，生成图片
!bash /content/scripts/test.sh